In [1]:
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine


In [120]:
df = pd.read_csv("c:/Users/fearn/Downloads/Jobs_NYC_Postings.csv")

In [121]:
pd.set_option('display.max_columns', None)
df =df.rename(columns={'Full-Time/Part-Time indicator': 'FT/PT', 'Minimum Qual Requirements' : 'Min Qualifications'})

In [122]:
df

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,FT/PT,Career Level,Salary Range From,Salary Range To,Salary Frequency,Work Location,Division/Work Unit,Job Description,Min Qualifications,Preferred Skills,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,589025,DEPARTMENT OF TRANSPORTATION,External,2,Supervisor Highway Repairer,SUPERVISOR HIGHWAY REPAIRER,Competitive-1,92472,00,Building Operations & Maintenance,F,Experienced (non-manager),399.68,399.68,Daily,55 Water St Ny Ny,Preventive Maintenance,Supervises a crew of Highway repairers and Ass...,Qualification Requirements Three years of f...,Ability to communicate effectively in verbal a...,The City of New York is an inclusive equal opp...,Resumes may be submitted electronically using ...,Hours: 0630 - 1500,Port Ivory,NaN,New York City residency is generally required ...,06/14/2023,NaN,06/21/2023,02/13/2024
1,552763,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Asset Management Program Planner,ASSOCIATE PROJECT MANAGER,Competitive-1,22427,02,"Engineering, Architecture, & Planning",F,Experienced (non-manager),74650.00,109409.00,Annual,96-05 Horace Harding Expway,ASSET MANAGEMENT PLANNING,IMPORTANT NOTE: Only those currently serving a...,Qualification Requirements One year of full-ti...,Knowledge/Skills/Abilities/ Work Experience: 1...,Appointments are subject to OMB approval. For ...,Click Apply Now,NaN,NaN,NaN,New York City Residency is not required for th...,09/29/2022,NaN,09/29/2022,02/13/2024
2,595336,HRA/DEPT OF SOCIAL SERVICES,External,7,CASE MANAGEMENT SUPERVISOR,ASSOCIATE BENEFITS OPPORTUNITY,Competitive-1,56316,01,Administration & Human Resources Social Services,F,Experienced (non-manager),56677.00,65179.00,Annual,95 Evergreen Ave Brooklyn,FIA Operations-NM,Family Independence Administration's (FIA) pro...,1. A four-year high school diploma or its educ...,â¢ Experience with Welfare Management System...,The Human Resources Administration/Department ...,APPLICANTS MUST BE PERMANENT IN THE JOB OPPORT...,Monday â Friday,NaN,NaN,New York City Residency is not required for th...,07/27/2023,NaN,07/27/2023,02/13/2024
3,597592,DEPT OF ENVIRONMENT PROTECTION,Internal,1,ASSISTANT PROJECT MANAGER,PROJECT MANAGER INTERN#,Competitive-1,22425,00,"Engineering, Architecture, & Planning Public S...",F,Experienced (non-manager),56313.00,64760.00,Annual,"465 Columbus Ave. Valhalla, Ny",UPSTATE CAPITAL CONSTR. - IFA,The NYC Department of Environmental Protection...,1. A baccalaureate degree from an accredited c...,â¢\tKnowledge of Microsoft Office Suite produ...,NaN,To apply click Apply Now DEP is an equal oppo...,NaN,"Sutton Park, 465 Columbus Avenue, Valhalla, NY...",NaN,New York City Residency is not required for th...,08/17/2023,NaN,08/17/2023,02/13/2024
4,607064,DEPARTMENT OF CITY PLANNING,Internal,1,Database Administrator,COMPUTER SPECIALIST (SOFTWARE),Competitive-1,13632,03,"Technology, Data & Innovation",F,Experienced (non-manager),115854.00,130701.00,Annual,"120 Broadway, New York, NY",ITD-Application Service Mgmt,THE AGENCY The Department of City Planning (DC...,(1) A baccalaureate degree from an accredited ...,"â¢\tBachelor's degree in Computer Science, In...",NaN,PLEASE NOTE: Only applicants under considerati...,NaN,NaN,NaN,New York City Residency is not required for th...,09/25/2023,NaN,10/06/2023,02/13/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,618573,BRONX DISTRICT ATTORNEY,External,1,Supervisor of Onboarding,ADMINISTRATIVE MANAGER,Competitive-1,10025,M2,Administration & Human Resources Communication...,F,Manager,95000.00,95000.00,Annual,198 E161St Street Bronx N.Y.,Support Staff,MUST BE AN ADMINISTRATIVE MANAGER TO APPLY LV ...,1. A baccalaureate degree from an accredited c...,NaN,NaN,"For City e

In [115]:
df['Career Level'].unique()

array(['Experienced (non-manager)', 'Student', 'Entry-Level', 'Manager',
       'Executive'], dtype=object)

In [123]:
df['Base Salary'] = 0

In [ ]:

df.loc[df['Salary Frequency'] == 'Daily', 'Base Salary'] = round(df['Salary Range From'].mul(210), 2)
df.loc[df['Salary Frequency'] == 'Annual', 'Base Salary'] = round(df['Salary Range From'], 2)

#calculate Base Salary for Hourly Employees dependent on their FT/PT Status
df['Base Salary'] = np.where((df['FT/PT'] == 'P') & (df['Salary Frequency'] == 'Hourly') , round(df['Salary Range From'].mul(20*52), 2), df['Base Salary'])
df['Base Salary'] = np.where((df['FT/PT'] == 'F') & (df['Salary Frequency'] == 'Hourly') , round(df['Salary Range From'].mul(40*52), 2), df['Base Salary'])

In [125]:
column_to_move = df.pop("Base Salary")  
df.insert(12, "Annual Base Salary", column_to_move) 

In [126]:
df = df.drop(['Title Classification', 'Title Code No', 'Salary Range To', 'Salary Range From', 'Salary Frequency', 'Hours/Shift', 'Post Until', 'Posting Updated', 'Process Date'], axis=1)

In [127]:
df = df.drop(['Preferred Skills', 'Additional Information', 'To Apply', 'Recruitment Contact', 'Work Location 1', 'Job Description'], axis=1)

In [128]:
df

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Level,Job Category,FT/PT,Career Level,Annual Base Salary,Work Location,Division/Work Unit,Min Qualifications,Residency Requirement,Posting Date
0,589025,DEPARTMENT OF TRANSPORTATION,External,2,Supervisor Highway Repairer,SUPERVISOR HIGHWAY REPAIRER,00,Building Operations & Maintenance,F,Experienced (non-manager),83932.8,55 Water St Ny Ny,Preventive Maintenance,Qualification Requirements Three years of f...,New York City residency is generally required ...,06/14/2023
1,552763,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Asset Management Program Planner,ASSOCIATE PROJECT MANAGER,02,"Engineering, Architecture, & Planning",F,Experienced (non-manager),74650.0,96-05 Horace Harding Expway,ASSET MANAGEMENT PLANNING,Qualification Requirements One year of full-ti...,New York City Residency is not required for th...,09/29/2022
2,595336,HRA/DEPT OF SOCIAL SERVICES,External,7,CASE MANAGEMENT SUPERVISOR,ASSOCIATE BENEFITS OPPORTUNITY,01,Administration & Human Resources Social Services,F,Experienced (non-manager),56677.0,95 Evergreen Ave Brooklyn,FIA Operations-NM,1. A four-year high school diploma or its educ...,New York City Residency is not required for th...,07/27/2023
3,597592,DEPT OF ENVIRONMENT PROTECTION,Internal,1,ASSISTANT PROJECT MANAGER,PROJECT MANAGER INTERN#,00,"Engineering, Architecture, & Planning Public S...",F,Experienced (non-manager),56313.0,"465 Columbus Ave. Valhalla, Ny",UPSTATE CAPITAL CONSTR. - IFA,1. A baccalaureate degree from an accredited c...,New York City Residency is not required for th...,08/17/2023
4,607064,DEPARTMENT OF CITY PLANNING,Internal,1,Database Administrator,COMPUTER SPECIALIST (SOFTWARE),03,"Technology, Data & Innovation",F,Experienced (non-manager),115854.0,"120 Broadway, New York, NY",ITD-Application Service Mgmt,(1) A baccalaureate degree from an accredited ...,New York City Residency is not required for th...,09/25/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5476,618573,BRONX DISTRICT ATTORNEY,External,1,Supervisor of Onboarding,ADMINISTRATIVE MANAGER,M2,Administration & Human Resources Communication...,F,Manager,95000.0,198 E161St Street Bronx N.Y.,Support Staff,1. A baccalaureate degree from an accredited c...,City Residency is not required for this position,12/04/2023
5477,620496,DEPT OF HEALTH/MENTAL HYGIENE,External,1,"First Line Supervisor, Bureau of Hepatitis, HI...",SUPERVISING PUBLIC HEALTH ADVI,00,Constituent Services & Community Programs Heal...,F,Experienced (non-manager),59301.0,42-09 28th Street,STI Programs,1. A baccalaureate degree from an accredited c...,New York City residency is generally required ...,12/19/2023
5478,623202,OFFICE OF THE COMPTROLLER,Internal,1,Research Support Intern for the Chief Investme...,SUMMER COLLEGE INTERN,00,"Finance, Accounting, & Procurement Policy, Res...",P,Student,16120.0,"1 Centre St., N.Y.",Asset Management,As of June of the Program year the prospective...,New York City residency is generally required ...,01/12/2024
5479,618020,DEPT OF ENVIRONMENT PROTECTION,External,1,Cost Estimating Manager,ADMINISTRATIVE ENGINEER,M3,"Engineering, Architecture, & Planning Policy, ...",F,Manager,72038.0,96-05 Horace Harding Expway,PROJECT MANAGEMENT OFFICE (PMO,1. A valid New York State License as a Profess...,New York City Residency is not required for th...,01/02/2024
